# Example 3: Post-Processing and Visualising Nonlinear Time-History Analysis Results using the "postprocessor" and "plotter" Classes

## Introduction


This Jupyter notebook is designed to postprocess and visualize thcloud analysis e resultfollowing a a **Nonlinear Time History AnalysiLs (NTHA)** conducted on a **Multi-Degree of Freedom (MDOF)*(after runningie Example in demos folder)s 

Thmain goals of this notebookok is:

**Result Post-Processing**: Extract and visualize critical response metrics to:
   - Visualise seismic demands such as peak storey drifts (PSD) and peak floor accelerations (PFA) along the building height 
   - Perform and visualise cloud analyses to characterise the engineering demand parameter (i.e., maximum peak storey drift) given intensity measure levels distribution (or EDP=MPSD|IM)
   - Estimate the median seismic intensities and total associated dispersions (i.e., accounting for record-to-record variability and modelling uncertainty) corresponding to user-defined demand-based damage thresholds 
   - Calculate and visualise damage probabilities (i.e., fragility functions) corresponding to distinct structural damage states

The notebook provides a step-by-step guide, covering each phase from MDOF model calibration, setup to input parameter configuration, analysis execution, and detailed results interpretation. Users should have some familiarity with python scripts, structural dynamics and computational modeling to fully benefit from this material.

## Initialize Libraries

In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

start = time.time()

# Set paths
vmtk = os.path.abspath(os.path.join("..",".."))
sys.path.append(vmtk)
os.chdir(os.path.join(vmtk,'src'))

# Import the classes necessary for postprocessing and visualising structural analysis results
from src.postprocessor import postprocessor
from src.plotter       import plotter
from src.utilities     import import_from_pkl

# Initialise the postprocessor and plotter classes
pp = postprocessor()
pl = plotter()

## Define Plotting Constants

In [ ]:
FONTSIZE_1 = 16
FONTSIZE_2 = 14
FONTSIZE_3 = 12

LINEWIDTH_1= 3
LINEWIDTH_2= 2
LINEWIDTH_3 = 1

RESOLUTION = 500

MARKER_SIZE_1 = 100
MARKER_SIZE_2 = 60
MARKER_SIZE_3 = 10

COLOR = "#399283"

## Required Input Parameters ##

In [ ]:
# Intensity measures to use for cloud analyses
IMTs      = ['PGA', 'SA(0.3s)', 'SA(0.6s)', 'SA(1.0s)','AvgSA']

# Damage thresholds (maximum peak storey drift values in rad)
damage_thresholds    =  [0.00150, 0.00545, 0.00952, 0.0135] 

# The lower limit to be applied for censoring edp values (below 0.1 the minimum threshold for slight damage is considered a negligible case)
lower_limit = 0.1*damage_thresholds[0]

# The upper limit to be applied for consoring edp values (above 1.5 the maximum threshold is considered a collapse case) 
censored_limit = 1.5*damage_thresholds[-1]   

# Define consequence model to relate structural damage to a decision variable (i.e., expected loss ratio) 
consequence_model = [0.05, 0.20, 0.60, 1.00] # damage-to-loss ratios

## Define Directories and Import Analysis Data ##

In [ ]:
# Define the directory of the ground-motion records and import the intensity measure pickle file containing all IMs (processed from example 1)
gm_directory  = f'{vmtk}/demos/in/records'            
ims           = import_from_pkl(f'{gm_directory}/imls_esrm20.pkl')     

# Define the main output directory and import the analysis output from a pickle file using the "import_from_pkl" function from "utilities"
nltha_directory = f'{vmtk}/demos/out/nltha' 
ansys_dict      = import_from_pkl(f'{nltha_directory}/ansys_out.pkl') # processed from example 2

## Post-Processing: Fragility Analysis on Cloud Data

#### The cloud analysis module (do_cloud_analysis) of the "postprocessor" class requires five mandatory arguments and one optional:
1. The intensity measure levels of the ground-motion records (imls)
2. The engineering demand parameters from the analysis (edps)
3. The demand-based damage thresholds (damage_thresholds)
4. The lower edp censoring limit (lower_limit)
5. The upper edp censoring limit (censored_limit)
6. OPTIONAL: The modelling uncertainty (sigma_build2build), default is set 0.3

In [ ]:
# Loop over the intensity measure types and perform cloud regression to fit the probabilistic seismic demand-capacity model
for _, current_imt in enumerate(IMTs):
    
    # Import the current intensity measure type
    imls = ims[f'{current_imt}']                   

    # Import the engineering demand parameters (i.e., mpsd) from the analysis dictionary (processed from example 2)
    edps = ansys_dict['mdof_max_peak_drift_list']  
    
    # Process cloud analysis results using the "do_cloud_analysis" function called from "postprocessor" 
    # The output will be automatically stored in a dictionary
    cloud_dict = pp.do_cloud_analysis(imls,
                                      edps,
                                      damage_thresholds,
                                      lower_limit,
                                      censored_limit) 
    
    ## Create a figures directory for each building class
    figures_directory = f'{vmtk}/example/out/nltha/figures'    
    if not os.path.exists(f'{figures_directory}'):
        os.makedirs(f'{figures_directory}')
    
    ## Visualise the cloud analysis results
    pl.plot_cloud_analysis(cloud_dict, 
                          figures_directory, 
                          plot_label = f'cloud_analysis_{current_imt}',
                          xlabel = f'{current_imt} [g]', 
                          ylabel = r'Maximum Peak Storey Drift, $\theta_{max}$ [%]') # The y-axis values of drift are converted to % automatically by the plotter

    ## Visualise the fragility functions
    pl.plot_fragility_analysis(cloud_dict,
                               figures_directory,
                               plot_label = f'fragility_{current_imt}',
                               xlabel = f'{current_imt}')

    ## Visualise the seismic demands
    pl.plot_demand_profiles(ansys_dict['mdof_peak_drift_list'], 
                            ansys_dict['mdof_peak_accel_list'], 
                            ansys_dict['control_nodes'], 
                            figures_directory,
                            plot_label="seismic_demand_profiles") # The y-axis values of drift and acceleration are converted to % and g automatically by the plotter
        
    ## Visualise the entire set of results using subplots
    pl.plot_ansys_results(cloud_dict,
                          ansys_dict['mdof_peak_drift_list'],
                          ansys_dict['mdof_peak_accel_list'],
                          ansys_dict['control_nodes'],
                          figures_directory,
                          plot_label = f'analysis_output_{current_imt}',
                          cloud_xlabel = f'{current_imt}',
                          cloud_ylabel = r'Maximum Peak Storey Drift, $\theta_{max}$ [%]')

## Post-Processing: Vulnerability Functions

In [ ]:
# To derive the vulnerability, the consequence model needs to convolved with the fragility functions. 
# To do so, we can use the "get_vulnerability_function" method from the "postprocessor" class.
structural_loss = pp.get_vulnerability_function(cloud_dict['poes'],
                                                consequence_model)

# Plot the vulnerability function to visualise
plt.plot(cloud_dict['intensities'], structural_loss, label = 'Structural Loss', color = COLOR, lw=LINEWIDTH_1)
plt.xlabel('Spectral Acceleration, Sa(T=1.0s) [g]', fontsize= FONTSIZE_1) # This is because the latest cloud_dict variable is associated with SA(1.0s)
plt.ylabel('Expected Loss Ratio', fontsize = FONTSIZE_1)
plt.legend(loc = 'lower right')
plt.grid(visible=True, which='major')
plt.grid(visible=True, which='minor')
plt.xlim([0.00, 5.00])
plt.show()

In [ ]:
end = time.time()
print('Elapsed Time:', (end - start)/60, 'minutes')